In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import json

categories_to_search = ['womens sport', 'bags', 
                        'fragrances', 'nails', 
                        'sunglasses', 'dresses', 
                        'jewellery', 'womens+shoes', 
                        'womens+watches']
products = []
item = 1

# Configurer Selenium pour utiliser Chrome
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Utiliser le mode sans tête (sans interface graphique)
options.add_argument("--disable-gpu")  # Désactiver le GPU pour éviter des erreurs
options.add_argument("--no-sandbox")  # Désactiver le sandbox pour éviter des erreurs

# Démarrer le navigateur Selenium
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

for category in categories_to_search:
    URL = f"https://www.amazon.com/s?k={category}+products"
    
    # Accéder à la page
    driver.get(URL)
    
    # Attendre que la page se charge complètement (ajuster le temps si nécessaire)
    time.sleep(5)  # Attendre 5 secondes
    
    # Récupérer le contenu de la page après le rendu dynamique par JavaScript
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Extraire les détails des produits
    for product in soup.find_all("div", {"data-component-type": "s-search-result"}):
        try:
            # Titre
            title = product.h2.text.strip() if product.h2 else "N/A"
            
            # Description (Amazon souvent avec une description d'image)
            description_tag = product.find("img", class_="s-image")
            description = description_tag["alt"] if description_tag else "N/A"
            
            # Prix
            price_whole = product.find("span", class_="a-price-whole")
            price_fraction = product.find("span", class_="a-price-fraction")
            if price_whole and price_fraction:
                price = f"{price_whole.text.strip()}.{price_fraction.text.strip()}"
            else:
                price = "N/A"
            
            # Évaluation
            rating_tag = product.find("span", class_="a-icon-alt")
            rating = rating_tag.text.strip() if rating_tag else "N/A"
            
            # Stock
            stock = "Available" if product.find("span", {"class": "a-color-price"}) else "UnAvailable"
            
            # Image
            image_tag = product.find("img", class_="s-image")
            image_url = image_tag["src"] if image_tag else "N/A"

            products.append({
                "id": item,
                "title": title,
                "description": description,
                "category": category,
                "price": price,
                "rating": rating,
                "stock": stock,
                "image": image_url,
                "reviews": []
            })
            item += 1

        except Exception as e:
            print(f"Erreur lors de l'extraction du produit : {e}")
    
    # Attendre avant de passer à la prochaine catégorie
    time.sleep(10)  # Attendre 10 secondes entre les requêtes

# Fermer le navigateur Selenium après l'exécution
driver.quit()

# Sauvegarder les données dans un fichier JSON
with open("products_selenium.json", "w", encoding="utf-8") as json_file:
    json.dump(products, json_file, ensure_ascii=False, indent=4)

print("Scraping terminé. Données sauvegardées dans products_selenium.json")


Scraping terminé. Données sauvegardées dans products_selenium.json
